In [1]:
from datasets import load_dataset
from transformers import AutoImageProcessor
import albumentations as A
from torch.utils.data import Dataset

# Load the dataset
ds = load_dataset("Francesco/animals-ij5d2")
train_ds = ds["train"]
val_ds = ds["validation"]
test_ds = ds["test"]
print(ds)
categories = train_ds.features["objects"].feature["category"].names
id2label = {index: x for index, x in enumerate(categories, start=0)}
label2id = {v: k for k, v in id2label.items()}
print(categories)
checkpoint = "PekingU/rtdetr_r50vd_coco_o365"
image_size = 480

image_processor = AutoImageProcessor.from_pretrained(
    checkpoint,
    do_resize=True,
    size={"width": image_size, "height": image_size},
)

train_augmentation_and_transform = A.Compose(
    [
        A.Perspective(p=0.1),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.1),
        A.Rotate(limit=20, p=0.5),
        A.RandomScale(scale_limit=0.2, p=0.5),
        A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5),
        A.Affine(scale=(0.9, 1.1), translate_percent=0.1, rotate=(-10, 10), shear=(-5, 5), p=0.5),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
        A.MotionBlur(blur_limit=3, p=0.2),
        A.GridDistortion(p=0.2),
    ],
    bbox_params=A.BboxParams(format="coco", label_fields=["category"], clip=True, min_area=25),
)

# to make sure boxes are clipped to image size and there is no boxes with area < 1 pixel
validation_transform = A.Compose(
    [A.NoOp()],
    bbox_params=A.BboxParams(format="coco", label_fields=["category"], clip=True, min_area=1),
)

class AnimalsDataset(Dataset):
    def __init__(self, dataset, image_processor, transform=None):
        self.dataset = dataset
        self.image_processor = image_processor
        self.transform = transform

    @staticmethod
    def format_image_annotations_as_coco(image_id, categories, boxes):
        """Format one set of image annotations to the COCO format

        Args:
            image_id (str): image id. e.g. "0001"
            categories (List[int]): list of categories/class labels corresponding to provided bounding boxes
            boxes (List[Tuple[float]]): list of bounding boxes provided in COCO format
                ([center_x, center_y, width, height] in absolute coordinates)

        Returns:
            dict: {
                "image_id": image id,
                "annotations": list of formatted annotations
            }
        """
        annotations = []
        for category, bbox in zip(categories, boxes):
            formatted_annotation = {
                "image_id": image_id,
                "category_id": category,
                "bbox": list(bbox),
                "iscrowd": 0,
                "area": bbox[2] * bbox[3],
            }
            annotations.append(formatted_annotation)

        return {
            "image_id": image_id,
            "annotations": annotations,
        }

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        image_id = sample["image_id"]
        image = sample["image"]
        boxes = sample["objects"]["bbox"]
        categories = sample["objects"]["category"]

        # Convert image to RGB numpy array
        image = np.array(image.convert("RGB"))

        # Apply augmentations
        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes, category=categories)
            image = transformed["image"]
            boxes = transformed["bboxes"]
            categories = transformed["category"]

        # Format annotations in COCO format for image_processor
        formatted_annotations = self.format_image_annotations_as_coco(image_id, categories, boxes)

        # Apply the image processor transformations: resizing, rescaling, normalization
        result = self.image_processor(
            images=image, annotations=formatted_annotations, return_tensors="pt"
        )

        # Image processor expands batch dimension, lets squeeze it
        result = {k: v[0] for k, v in result.items()}

        return result

train_ds = AnimalsDataset(train_ds, image_processor, transform=train_augmentation_and_transform)
val_ds = AnimalsDataset(val_ds, image_processor, transform=validation_transform)
test_ds = AnimalsDataset(test_ds, image_processor, transform=validation_transform)

2024-11-04 10:39:45.278515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-04 10:39:45.295206: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-04 10:39:45.300462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-04 10:39:45.313165: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-04 10:39:46.147347: W tensorflow/compiler/tf2

DatasetDict({
    train: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 100
    })
    test: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 200
    })
})
['animals', 'cat', 'chicken', 'cow', 'dog', 'fox', 'goat', 'horse', 'person', 'racoon', 'skunk']


import numpy as np
from PIL import Image, ImageDraw

# Get mapping from category id to category name


print(categories)
print(id2label)
print(label2id)

# Load image and annotations
image = train_ds[2]["image"]
annotations = train_ds[2]["objects"]

# Draw bounding boxes and labels
draw = ImageDraw.Draw(image)
for i in range(len(annotations["id"])):
    box = annotations["bbox"][i]
    class_idx = annotations["category"][i]
    x, y, w, h = tuple(box)
    draw.rectangle((x, y, x + w, y + h), outline="red", width=1)
    draw.text((x, y), id2label[class_idx], fill="white")

image




for i in [15, 16, 17]:
    image = train_ds[i]["image"]
    annotations = train_ds[i]["objects"]

    # Apply the augmentation
    output = train_augmentation_and_transform(image=np.array(image), bboxes=annotations["bbox"], category=annotations["category"])

    # Unpack the output
    image = Image.fromarray(output["image"])
    categories, boxes = output["category"], output["bboxes"]

    # Draw the augmented image
    draw = ImageDraw.Draw(image)
    for category, box in zip(categories, boxes):
        x, y, w, h = box
        draw.rectangle((x, y, x + w, y + h), outline="red", width=1)
        draw.text((x, y), id2label[category], fill="white")
    image.show()

for i in [15, 16, 17]:
    sample = train_ds[i]

    # De-normalize image
    image = sample["pixel_values"]
    print("Image tensor shape:", image.shape)
    image = image.numpy().transpose(1, 2, 0)
    image = (image - image.min()) / (image.max() - image.min()) * 255.
    image = Image.fromarray(image.astype(np.uint8))

    # Convert boxes from [center_x, center_y, width, height] to [x, y, width, height] for visualization
    boxes = sample["labels"]["boxes"].numpy()
    print("Boxes shape:", boxes.shape)
    boxes[:, :2] = boxes[:, :2] - boxes[:, 2:] / 2
    w, h = image.size
    boxes = boxes * np.array([w, h, w, h])[None]

    categories = sample["labels"]["class_labels"].numpy()
    print("Categories shape:", categories.shape)

    # Draw boxes and labels on image
    draw = ImageDraw.Draw(image)
    for box, category in zip(boxes, categories):
        x, y, w, h = box
        draw.rectangle([x, y, x + w, y + h], outline="red", width=1)
        draw.text((x, y), id2label[category], fill="white")
    image.show()

In [2]:
from transformers.image_transforms import center_to_corners_format
import torch
import numpy as np
from dataclasses import dataclass
from torchmetrics.detection.mean_ap import MeanAveragePrecision

def collate_fn(batch):
    data = {}
    data["pixel_values"] = torch.stack([x["pixel_values"] for x in batch])
    data["labels"] = [x["labels"] for x in batch]
    return data

def convert_bbox_yolo_to_pascal(boxes, image_size):
    """
    Convert bounding boxes from YOLO format (x_center, y_center, width, height) in range [0, 1]
    to Pascal VOC format (x_min, y_min, x_max, y_max) in absolute coordinates.

    Args:
        boxes (torch.Tensor): Bounding boxes in YOLO format
        image_size (Tuple[int, int]): Image size in format (height, width)

    Returns:
        torch.Tensor: Bounding boxes in Pascal VOC format (x_min, y_min, x_max, y_max)
    """
    # convert center to corners format
    boxes = center_to_corners_format(boxes)

    # convert to absolute coordinates
    height, width = image_size
    boxes = boxes * torch.tensor([[width, height, width, height]])

    return boxes

@dataclass
class ModelOutput:
    logits: torch.Tensor
    pred_boxes: torch.Tensor


class MAPEvaluator:

    def __init__(self, image_processor, threshold=0.00, id2label=None):
        self.image_processor = image_processor
        self.threshold = threshold
        self.id2label = id2label

    def collect_image_sizes(self, targets):
        """Collect image sizes across the dataset as list of tensors with shape [batch_size, 2]."""
        image_sizes = []
        for batch in targets:
            batch_image_sizes = torch.tensor(np.array([x["size"] for x in batch]))
            image_sizes.append(batch_image_sizes)
        return image_sizes

    def collect_targets(self, targets, image_sizes):
        post_processed_targets = []
        for target_batch, image_size_batch in zip(targets, image_sizes):
            for target, size in zip(target_batch, image_size_batch):
                boxes = torch.tensor(target["boxes"])
                boxes = convert_bbox_yolo_to_pascal(boxes, size)
                labels = torch.tensor(target["class_labels"])
                post_processed_targets.append({"boxes": boxes, "labels": labels})
        return post_processed_targets

    def collect_predictions(self, predictions, image_sizes):
        post_processed_predictions = []
        for batch, target_sizes in zip(predictions, image_sizes):
            batch_logits, batch_boxes = batch[1], batch[2]
            output = ModelOutput(logits=torch.tensor(batch_logits), pred_boxes=torch.tensor(batch_boxes))
            post_processed_output = self.image_processor.post_process_object_detection(
                output, threshold=self.threshold, target_sizes=target_sizes
            )
            post_processed_predictions.extend(post_processed_output)
        return post_processed_predictions

    @torch.no_grad()
    def __call__(self, evaluation_results):

        predictions, targets = evaluation_results.predictions, evaluation_results.label_ids

        image_sizes = self.collect_image_sizes(targets)
        post_processed_targets = self.collect_targets(targets, image_sizes)
        post_processed_predictions = self.collect_predictions(predictions, image_sizes)

        evaluator = MeanAveragePrecision(box_format="xyxy", class_metrics=True)
        evaluator.warn_on_many_detections = False
        evaluator.update(post_processed_predictions, post_processed_targets)

        metrics = evaluator.compute()

        # Replace list of per class metrics with separate metric for each class
        classes = metrics.pop("classes")
        map_per_class = metrics.pop("map_per_class")
        mar_100_per_class = metrics.pop("mar_100_per_class")
        for class_id, class_map, class_mar in zip(classes, map_per_class, mar_100_per_class):
            class_name = id2label[class_id.item()] if id2label is not None else class_id.item()
            metrics[f"map_{class_name}"] = class_map
            metrics[f"mar_100_{class_name}"] = class_mar

        metrics = {k: round(v.item(), 4) for k, v in metrics.items()}

        return metrics

eval_compute_metrics_fn = MAPEvaluator(image_processor=image_processor, threshold=0.01, id2label=id2label)

In [3]:
from transformers import TrainingArguments, Trainer, AutoModelForObjectDetection

model = AutoModelForObjectDetection.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
    anchor_image_size=None,
    ignore_mismatched_sizes=True,
)

training_args = TrainingArguments(
    output_dir="rtdetr_model",
    num_train_epochs=100,
    max_grad_norm=0.1,
    learning_rate=1e-5,
    optim="adamw_torch_fused",
    weight_decay=0.10,
    gradient_accumulation_steps=2,
    lr_scheduler_type="cosine", 
    warmup_steps=300,
    per_device_train_batch_size=8,
    dataloader_num_workers=6,
    metric_for_best_model="eval_map",
    greater_is_better=True,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=4,
    remove_unused_columns=False,
    eval_do_concat_batches=False,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    data_collator=collate_fn,
    compute_metrics=eval_compute_metrics_fn,
)

trainer.train()

Could not load the custom kernel for multi-scale deformable attention: Error building extension 'MultiScaleDeformableAttention': [1/2] /usr/local/cuda-11.5/bin/nvcc --generate-dependencies-with-compile --dependency-output ms_deform_attn_cuda.cuda.o.d -DTORCH_EXTENSION_NAME=MultiScaleDeformableAttention -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -I/home/eagle/anaconda3/lib/python3.11/site-packages/transformers/kernels/deformable_detr -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include/TH -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include/THC -isystem /usr/local/cuda-11.5/include -isystem /home/eagle/anaconda3/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ 

Epoch,Training Loss,Validation Loss,Map,Map 50,Map 75,Map Small,Map Medium,Map Large,Mar 1,Mar 10,Mar 100,Mar Small,Mar Medium,Mar Large,Map Animals,Mar 100 Animals,Map Cat,Mar 100 Cat,Map Chicken,Mar 100 Chicken,Map Cow,Mar 100 Cow,Map Dog,Mar 100 Dog,Map Fox,Mar 100 Fox,Map Goat,Mar 100 Goat,Map Horse,Mar 100 Horse,Map Person,Mar 100 Person,Map Racoon,Mar 100 Racoon,Map Skunk,Mar 100 Skunk
1,No log,174.083694,0.024300,0.029100,0.027500,0.000000,0.000000,0.043200,0.091600,0.136500,0.213300,0.000000,0.000000,0.214600,-1.000000,-1.000000,0.026100,0.614300,0.020300,0.186700,0.000000,0.052400,0.185600,0.441200,0.009400,0.488900,0.000000,0.000000,0.000200,0.212500,0.000000,0.024000,0.000000,0.000000,0.001800,0.113300
2,No log,146.844086,0.027400,0.031800,0.029700,0.000000,0.000000,0.053700,0.126800,0.209700,0.239200,0.000000,0.000000,0.239900,-1.000000,-1.000000,0.041500,0.657100,0.034300,0.226700,0.000000,0.028600,0.186100,0.558800,0.010300,0.566700,0.000000,0.000000,0.000500,0.262500,0.000000,0.032000,0.000000,0.000000,0.000900,0.060000
3,No log,109.644470,0.047500,0.055500,0.051300,0.000000,0.000000,0.063700,0.144700,0.249400,0.283600,0.000000,0.000000,0.285800,-1.000000,-1.000000,0.121900,0.557100,0.035100,0.233300,0.000100,0.090500,0.288100,0.594100,0.018400,0.711100,0.000000,0.000000,0.000500,0.187500,0.000300,0.116000,0.000000,0.000000,0.010100,0.346700
4,No log,88.634216,0.088700,0.101800,0.100000,0.000000,0.000000,0.102900,0.206700,0.340500,0.382100,0.000000,0.000000,0.384300,-1.000000,-1.000000,0.508100,0.700000,0.007300,0.413300,0.001600,0.095200,0.286500,0.605900,0.050100,0.688900,0.000400,0.058600,0.001800,0.487500,0.000600,0.160000,0.000300,0.118200,0.030700,0.493300
5,No log,68.970810,0.108400,0.123400,0.115500,0.000000,0.000000,0.124900,0.348700,0.559500,0.611500,0.000000,0.000000,0.623100,-1.000000,-1.000000,0.340500,0.928600,0.010700,0.640000,0.004200,0.490500,0.470400,0.852900,0.052800,0.822200,0.006900,0.262100,0.011500,0.587500,0.083700,0.720000,0.000800,0.118200,0.102500,0.693300
6,No log,53.694229,0.169700,0.195400,0.183100,0.000000,0.000700,0.179200,0.486400,0.703200,0.769200,0.000000,0.166700,0.783600,-1.000000,-1.000000,0.277500,0.914300,0.027700,0.926700,0.013200,0.738100,0.522500,0.935300,0.249700,0.877800,0.036200,0.593100,0.046000,0.725000,0.344900,0.768000,0.002200,0.327300,0.177300,0.886700
7,No log,40.944603,0.239800,0.282900,0.259700,0.000000,0.000000,0.249200,0.551500,0.747900,0.795700,0.000000,0.000000,0.812400,-1.000000,-1.000000,0.385900,0.900000,0.060400,0.873300,0.040800,0.709500,0.588200,0.911800,0.272400,0.866700,0.123200,0.762100,0.083200,0.800000,0.640400,0.808000,0.004500,0.372700,0.199400,0.953300
8,No log,32.074699,0.316600,0.378800,0.348900,0.000000,0.000000,0.337300,0.559700,0.786500,0.834800,0.000000,0.000000,0.851900,-1.000000,-1.000000,0.553900,0.914300,0.089300,0.900000,0.197700,0.728600,0.654600,0.935300,0.414700,0.933300,0.132900,0.737900,0.139200,0.787500,0.720300,0.844000,0.010100,0.600000,0.253400,0.966700
9,No log,26.869062,0.409000,0.483900,0.443600,0.000000,0.000000,0.434000,0.605100,0.835700,0.868900,0.000000,0.000000,0.886400,-1.000000,-1.000000,0.610200,0.914300,0.364300,0.880000,0.388300,0.742900,0.674600,0.923500,0.502700,0.922200,0.196700,0.869000,0.145200,0.787500,0.715800,0.860000,0.043200,0.836400,0.449000,0.953300
10,No log,24.118023,0.477700,0.582900,0.527000,0.000000,0.016800,0.500100,0.595700,0.815200,0.854600,0.000000,0.066700,0.870700,-1.000000,-1.000000,0.809000,0.928600,0.302300,0.886700,0.423100,0.733300,0.688900,0.929400,0.600700,0.866700,0.206300,0.855200,0.174000,0.762500,0.718700,0.828000,0.150500,0.809100,0.703300,0.946700


There were missing keys in the checkpoint model loaded: ['class_embed.0.weight', 'class_embed.0.bias', 'class_embed.1.weight', 'class_embed.1.bias', 'class_embed.2.weight', 'class_embed.2.bias', 'class_embed.3.weight', 'class_embed.3.bias', 'class_embed.4.weight', 'class_embed.4.bias', 'class_embed.5.weight', 'class_embed.5.bias', 'bbox_embed.0.layers.0.weight', 'bbox_embed.0.layers.0.bias', 'bbox_embed.0.layers.1.weight', 'bbox_embed.0.layers.1.bias', 'bbox_embed.0.layers.2.weight', 'bbox_embed.0.layers.2.bias', 'bbox_embed.1.layers.0.weight', 'bbox_embed.1.layers.0.bias', 'bbox_embed.1.layers.1.weight', 'bbox_embed.1.layers.1.bias', 'bbox_embed.1.layers.2.weight', 'bbox_embed.1.layers.2.bias', 'bbox_embed.2.layers.0.weight', 'bbox_embed.2.layers.0.bias', 'bbox_embed.2.layers.1.weight', 'bbox_embed.2.layers.1.bias', 'bbox_embed.2.layers.2.weight', 'bbox_embed.2.layers.2.bias', 'bbox_embed.3.layers.0.weight', 'bbox_embed.3.layers.0.bias', 'bbox_embed.3.layers.1.weight', 'bbox_embed.3.l

TrainOutput(global_step=4400, training_loss=22.407134898792613, metrics={'train_runtime': 4330.7907, 'train_samples_per_second': 16.163, 'train_steps_per_second': 1.016, 'total_flos': 1.240948138752e+19, 'train_loss': 22.407134898792613, 'epoch': 100.0})

In [ ]:
import torch
import requests
from PIL import Image, ImageDraw
from pprint import pprint

metrics = trainer.evaluate(eval_dataset=test_ds, metric_key_prefix="eval")
pprint(metrics)
device = "cuda"

image = Image.open("test.jpg")

inputs = image_processor(images=[image], return_tensors="pt")
inputs = inputs.to(device)
with torch.no_grad():
    outputs = model(**inputs)
target_sizes = torch.tensor([image.size[::-1]])

result = image_processor.post_process_object_detection(outputs, threshold=0.3, target_sizes=target_sizes)[0]

for score, label, box in zip(result["scores"], result["labels"], result["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

image_with_boxes = image.copy()
draw = ImageDraw.Draw(image_with_boxes)

for score, label, box in zip(result["scores"], result["labels"], result["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    x, y, x2, y2 = tuple(box)
    draw.rectangle((x, y, x2, y2), outline="red", width=1)
    draw.text((x, y), model.config.id2label[label.item()], fill="white")

image_with_boxes

In [12]:
model.save_pretrained("decent_model")
image_processor.save_pretrained("processor")

['processor/preprocessor_config.json']

In [2]:
import cv2
import torch
from transformers import AutoModelForObjectDetection, AutoImageProcessor
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(categories)
# Load the trained model and the image processor
checkpoint_path = "checkpoint-8750"
model = AutoModelForObjectDetection.from_pretrained(
    checkpoint_path,
    id2label=id2label,
    label2id=label2id,
    anchor_image_size=None,
    ignore_mismatched_sizes=True
)
processor = AutoImageProcessor.from_pretrained(checkpoint_path)
model.to(device)
# Open the camera (0 is the default camera, change if you have multiple cameras)
cap = cv2.VideoCapture(0)

# Set the camera resolution (optional)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)

while True:
    ret, frame = cap.read()  # Capture frame-by-frame

    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame to RGB (OpenCV uses BGR by default)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Preprocess the image
    inputs = processor(images=rgb_frame, return_tensors="pt").to(device)

    # Run inference on the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Post-process the outputs to get boxes, labels, and scores
    target_sizes = torch.tensor([rgb_frame.shape[:2]]).to(device)  # Use the original image size for scaling
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes)[0]

    # Extract the boxes, labels, and scores
    boxes = results['boxes'].cpu().numpy()  # Bounding boxes
    labels = results['labels'].cpu().numpy()  # Predicted class labels
    scores = results['scores'].cpu().numpy()  # Confidence scores

    # Draw the bounding boxes on the frame
    for box, label, score in zip(boxes, labels, scores):
        if score > 0.3:  # Only show boxes with a confidence score > 0.5
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Draw rectangle

            # Display label and score
            label_text = f"{id2label[label]}: {score:.2f}"
            cv2.putText(frame, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Display the frame with bounding boxes
    cv2.imshow('Camera Feed - Object Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()

['animals', 'cat', 'chicken', 'cow', 'dog', 'fox', 'goat', 'horse', 'person', 'racoon', 'skunk']


Could not load the custom kernel for multi-scale deformable attention: Error building extension 'MultiScaleDeformableAttention': [1/2] /usr/local/cuda-11.5/bin/nvcc --generate-dependencies-with-compile --dependency-output ms_deform_attn_cuda.cuda.o.d -DTORCH_EXTENSION_NAME=MultiScaleDeformableAttention -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -I/home/eagle/anaconda3/lib/python3.11/site-packages/transformers/kernels/deformable_detr -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include/TH -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include/THC -isystem /usr/local/cuda-11.5/include -isystem /home/eagle/anaconda3/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ 

KeyboardInterrupt: 

In [2]:
import cv2
import torch
import time
from transformers import AutoModelForObjectDetection, AutoImageProcessor
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model and the image processor
checkpoint_path = "checkpoint-8750"
model = AutoModelForObjectDetection.from_pretrained(
    checkpoint_path,
    id2label=id2label,
    label2id=label2id,
    anchor_image_size=None,
    ignore_mismatched_sizes=True
)
processor = AutoImageProcessor.from_pretrained(checkpoint_path)
model.to(device)

# Open the camera (0 is the default camera, change if you have multiple cameras)
cap = cv2.VideoCapture(0)

# Set the camera resolution (optional)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)

while True:
    start_time = time.time()  # Start time for FPS calculation
    
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame to RGB (OpenCV uses BGR by default)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Preprocess the image
    inputs = processor(images=rgb_frame, return_tensors="pt").to(device)

    # Run inference on the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Post-process the outputs to get boxes, labels, and scores
    target_sizes = torch.tensor([rgb_frame.shape[:2]]).to(device)  # Use the original image size for scaling
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes)[0]

    # Extract the boxes, labels, and scores
    boxes = results['boxes'].cpu().numpy()  # Bounding boxes
    labels = results['labels'].cpu().numpy()  # Predicted class labels
    scores = results['scores'].cpu().numpy()  # Confidence scores

    # Draw the bounding boxes on the frame
    for box, label, score in zip(boxes, labels, scores):
        if score > 0.3:  # Only show boxes with a confidence score > 0.3
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Draw rectangle

            # Display label and score
            label_text = f"{id2label[label]}: {score:.2f}"
            cv2.putText(frame, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Calculate FPS
    end_time = time.time()
    fps = 1 / (end_time - start_time)
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame with bounding boxes
    cv2.imshow('Camera Feed - Object Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()


Could not load the custom kernel for multi-scale deformable attention: Error building extension 'MultiScaleDeformableAttention': [1/2] /usr/local/cuda-11.5/bin/nvcc --generate-dependencies-with-compile --dependency-output ms_deform_attn_cuda.cuda.o.d -DTORCH_EXTENSION_NAME=MultiScaleDeformableAttention -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -I/home/eagle/anaconda3/lib/python3.11/site-packages/transformers/kernels/deformable_detr -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include/TH -isystem /home/eagle/anaconda3/lib/python3.11/site-packages/torch/include/THC -isystem /usr/local/cuda-11.5/include -isystem /home/eagle/anaconda3/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ 

In [5]:
print(model)

RTDetrForObjectDetection(
  (model): RTDetrModel(
    (backbone): RTDetrConvEncoder(
      (model): RTDetrResNetBackbone(
        (embedder): RTDetrResNetEmbeddings(
          (embedder): Sequential(
            (0): RTDetrResNetConvLayer(
              (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (1): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (2): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
          )
          (pooler): MaxPool2d(